# Ziel des Notebooks
* Wir validieren, ob wir die BirdNET Python Library so zum Laufen kriegen, dass sie:

    * Exakt (oder fast exakt) die gleichen Ergebnisse liefert wie deine Birdnet-Analyzer-App.

    * Auf deiner CPU schnell genug läuft.

* Relevante Funktionen aus der Dokumentation nutzen

In [ ]:
from pathlib import Path

Path.cwd().parent

In [ ]:
import os, sys, platform, json, time
from pathlib import Path

print("Python:", sys.version)
print("Platform:", platform.platform())
print("CWD:", Path.cwd())

PROJECT_ROOT = Path.cwd().parent  # passe an, falls du im notebooks/ Ordner bist
DATA_DIR = PROJECT_ROOT / "data" / "audio_raw"   # anpassen
OUT_DIR = PROJECT_ROOT / "outputs" / "experiments" / "birdnet"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR exists:", DATA_DIR.exists(), DATA_DIR)
print("OUT_DIR:", OUT_DIR)

In [ ]:
import birdnet
